In [ ]:
# OS/ User specific 
import os
# DATASET_PATH = "/home/jcharlet/workspace/datascience/machine_learning_notes/data/"
DATASET_PATH = "/opt/dataiku-dss/config/ipython_notebooks/JCKAGGLEHOUSEPRICES/data/" 
# DATASET_PATH = os.path.abspath(os.path.join( os.getcwd() , 'data')) + r'\'

### Kaggle project - Table of Contents
* 1 - Define the problem
* [Load data and displaying info](#load-data)
* 2 - Prepare Data
    * [Identify features](#identify)
        * Separate numerical from categorical features
        * Separate nominal and ordinal (from categorical features)
    * [Clean data](#clean)
        * Remove numerical features with missing values
        * Remove categorical features with missing values
        * drop outliers in numerical values # WIP
    * [transform](#transform) # TODO
        * transform categorical values #TODO
* 2 - [Feature selection](#feature-selection) #WIP
    * Select features using random forest classifier #WIP
    
* 3 - [Spot Check Algorithms](#spot-check)
    * split dataset
    * train on multiple algorithms  # WIP
    
    
## 7/8: next step is to work on categorical data

# Define the problem

From project https://www.kaggle.com/c/house-prices-advanced-regression-techniques

- Step 1: What is the problem? 

house prices are hard to predict, we want to know how much my beautiful flat is worth on the market in the USA

- Step 2: Why does the problem need to be solved? 

MONEY MONEY
and especially we need to learn Machine Learning

- Step 3: How would I solve the problem? 

housing agents do statistical studies.

Data pre-processing is very important. We have a lot of features, dirty data (missing values), numerical/ordinal/nominal values.

We'll follow the process from https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

- 1 - Define the Problem
- 2 - Prepare Data
- 3 - Spot Check Algorithms
- 4 - Improve Results
- 5 - Present Results

# Load data <a class="anchor" id="load-data"></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### PLOTS
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D

### Split dataset
from sklearn.model_selection import train_test_split


#training
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 

# standardization
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# training evaluation
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv(DATASET_PATH + "train.csv")
df_test = pd.read_csv(DATASET_PATH + "test.csv")


In [ ]:
print(df.info())


# Prepare data <a class="anchor" id="clean-data"></a>
## Identify features<a class="anchor" id="identify"></a>
### Separate numerical from categorical features

In [ ]:
# Separating data features according to data type: numeric or string

X_numeric_labels = set(df._get_numeric_data().columns.tolist())
X_categorical_labels = set(df.columns.tolist()).difference(X_numeric_labels)

In [ ]:
print('These are numerical features {} \n'.format(X_numeric_labels))
print('These are categorical features {} \n'.format(X_categorical_labels))


### Separate nominal and ordinal (from categorical features)

In [ ]:
# Separating categorical features in two sets: nominal and ordinal
# The values in features asociated with categorical ordinal feautures are: [Ex, Gd, GLQ, GdPrv, Fin] ... etc

categorical_ordinal_list_values = ['Ex', 'Gd', 'GLQ', 'GdPrv', 'Fin'] # There are not all of the values. Here must be
# included all the possible values for the categorical ordinal features

X_categorical_nominal_labels = (df[list(X_categorical_labels)].isin(categorical_ordinal_list_values) == False).all().loc[lambda df: df.values == True].axes

# The previous instruction returns the index of the categorical_nominal_values. I think we could include the following features:
#LandSlope has values: Gtl -> Gentle slopen; Mod -> Moderate Slope; Sev -> Severe Slope
#Functional has values: Typ -> Typical Functionality; Min1 -> Minor Deductions 1; Min2 -> Minor Deductions 2; Mod -> Moderate Deductions; Maj1 -> Major Deductions 1; Maj2 -> Major Deductions 2; Sev -> Severely Damaged; Sal -> Salvage only
#PavedDrive has values:   Y -> Paved ; P -> Partial Pavement; N -> Dirt/Gravel



In [ ]:

print(categorical_ordinal_list_values)
print()
print(X_categorical_nominal_labels[0].tolist())

### Remarks
<p> From the set of set of **X_categorical_labels** the set of **X_categorical_nominal_labels** is extracted to get 
the set of **X_categorical_ordinal_labels**</p>

* On the set of **X_categorical_ordinal_labels** we can apply pag 104 **mapping strategy**

* On the set of **X_categorical_nominal_labels** we can apply **one-hot encoding strategy** pag 106

In [ ]:
X_categorical_ordinal_labels = set(X_categorical_labels).difference(set(X_categorical_nominal_labels[0].tolist()))

In [ ]:
# print(X_categorical_labels)

In [ ]:
# df[list(X_categorical_ordinal_labels)]

## Clean data<a class="anchor" id="clean"></a>

#### Some considerations


* <p> Page 107 of "our book":  "After executing the preceding code, the first column of the NumPy array X now holds the new
colour values, which are encoded as follows:
blue to 0
green to 1
red to 2
If we stop at this point and feed the array to our classifier, *we will make one of the most common 
mistakes in dealing with categorical data*. Can you spot the problem? *
Although the colour values do not come in any particular order, a learning algorithm will now assume
that green is larger than blue, and red is larger than green. 
Although this assumption is incorrect, the algorithm could still produce useful results. 
However, those results would not be optimal* "</p>

* <p> In the numerical feature variables there is the possibility that there are categorical numerical features variables. How to identify this kinf of features ? 
 I think the main problem is what strategy to use to replace values ? median, mode or mean ? </p>
 

In [ ]:
df_cleaned=df.copy()

### Remove categorical features with missing values

In [ ]:
# we are dropping any categorical feature with at least one null value, we can relax this requirement by changing df > 0 to 
# df > n with n > 0

X_categorical_missing_values_labels = \
        df[list(X_categorical_labels)].isnull().sum().loc[lambda df: df > 0].index.tolist()
X_categorical_not_missing_values_labels=[x for x in X_categorical_labels if x not in X_categorical_missing_values_labels]

print(X_categorical_missing_values_labels)

print(" Number of Total Categorical Values = {} \n Number of Categorical Missing Values = {}  \n Difference = {}"\
      .format(len(X_categorical_labels), len(X_categorical_missing_values_labels), \
              len(X_categorical_labels)- len(X_categorical_missing_values_labels) ))

df_cleaned.drop(X_categorical_missing_values_labels, 1,inplace=True)

### Remove numerical features with missing values

In [ ]:
X_numerical_missing_values_labels=df_cleaned[df_cleaned._get_numeric_data().columns.tolist()].isnull().sum()\
                    .loc[lambda df: df.values > 0].index.tolist()
X_numerical_not_missing_values_labels=[x for x in X_numeric_labels if x not in X_numerical_missing_values_labels]

print(" Number of Total Numerical Values = {} \n Number of Numerical Missing Values = {}  \n Difference = {}"\
      .format(len(X_numeric_labels), len(X_numerical_missing_values_labels), \
              len(X_numeric_labels)- len(X_numerical_missing_values_labels) ))


df_cleaned.drop(X_numerical_missing_values_labels, 1,inplace=True)

In [ ]:
# print(X_numerical_missing_values_labels)
# print(categorical_missing_values_labels)
# df_cleaned.columns.values
# df_cleaned.head()

### Drop outliers in numerical values # WIP

In [ ]:
# def drop_outliers(df, field_name):
#     distance = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
#     df.drop(df[df[field_name] > distance + np.percentile(df[field_name], 75)].index, inplace=True)
#     df.drop(df[df[field_name] < np.percentile(df[field_name], 25) - distance].index, inplace=True)

# # drop_outliers(df_cleaned, 'LotArea')    
# df_cleaned_no_outlier = df.copy()
# for feature in X_numerical_not_missing_values_labels:
#     drop_outliers(df_cleaned_no_outlier, feature)    

In [ ]:
# print("nb of rows with outliers:{}, without: {}".format(df.shape[0],df_cleaned_no_outlier.shape[0]))


We are currently removing far too many items.

attempt above was removing values using univariate method: unsatisfying: we remove a third of the dataset, we should rather use filters here if we really want to deal with outliers on 1 dimension

but a much better way would be to find outliers in the whole dataset on all dimensions using multivariate method. We would then remove those outliers from the training set to improve accuracy

see http://scikit-learn.org/stable/modules/outlier_detection.html
http://scikit-learn.org/stable/auto_examples/covariance/plot_outlier_detection.html

## Transform<a class="anchor" id="transform"></a>

In [ ]:
df_cleaned.set_index("Id")
dfTransformed=pd.get_dummies(df_cleaned.copy())
dfTransformed.drop("Id",axis=1,inplace=True)

In [ ]:
dfTransformed.head()


# Feature Selection<a class="anchor" id="feature-selection"></a>

In [ ]:
# feature_columns=filter(lambda x: x !="SalePrice" and x!="Id", df.columns)
# feature_numerical_columns = df[feature_columns].select_dtypes(include=[np.number]).columns.tolist()
feature_columns=[x for x in dfTransformed if x !="SalePrice"]
# feature_columns=X_numerical_not_missing_values_labels
# feature_columns.extend(X_categorical_not_missing_values_labels)
prediction_column="SalePrice"
# # print(feature_columns)
X = dfTransformed[feature_columns].values
Y=dfTransformed[[prediction_column]].values;

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier();
model = model.fit(X, Y)    
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("label {} : {}".format(feature_columns[f],importances[indices[f]]))
# print(indices)

# plt.bar(range(X.shape[1]),importances[indices],align='center')
# plt.xticks(range(X.shape[1]),feature_numerical_columns,rotation=90);
# plt.rcParams["figure.figsize"] = (17,5)


To draw graph and select features

# Spot Check Algorithms<a class="anchor" id="spot-check"></a>

## Useful functions

In [ ]:
def splitDataSet(df,feature_columns):
    X = df[feature_columns].values
    Y=df["SalePrice"].values;
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33,random_state=42)
    # print(Y_train.head())
    # X_train.head()
    return X_train,X_test,Y_train,Y_test


def trainOnMultipleModels(X_train,X_test,Y_train,Y_test,models,modelsWhichNeedNormalization):
    
    print("5 expected values: {}\n".format(Y_test[0:5].tolist()))
    containers=[]
    for name,model in models:
    #     Train the model using the training sets
        model.fit(X_train, Y_train)

        # Make predictions using the testing set
        y_pre = model.predict(X_test)
        mean_squared_err= mean_squared_error(Y_test, y_pre)
        print("Training on {}\n   mean squared log error: {}\n".format(name,mean_squared_err))
        print("   5 predicted values: {}\n".format(y_pre[0:5]))
        containers.append([name,model,y_pre,mean_squared_err])

#     print("5 expected values: {}\n".format(Y_test[0:5].tolist()))
    for name,model in modelsWhichNeedNormalization:
    #     Train the model using the training sets
        sc=StandardScaler()
        sc.fit(X_train)

        X_train_scaled = sc.transform(X_train)
        model.fit(X_train_scaled, Y_train)

        # Make predictions using the testing set
        X_test_scaled = sc.transform(X_test)
        y_pre = model.predict(X_test_scaled)

        mean_squared_err= mean_squared_error(Y_test, y_pre)
        print("Training on {}\n   mean squared log error: {}\n".format(name,mean_squared_err))
        print("   5 predicted values: {}\n".format(y_pre[0:5]))
        containers.append([name,model,y_pre,mean_squared_err])
    return containers

def displayGraphTrueVsPred(name,y_pre,Y_test):
    plt.rcParams["figure.figsize"] = (7,7)

    preds_df=pd.DataFrame(data=y_pre,columns=["predictions"])
    preds_df["true values"]=Y_test

    print("Model: {}".format(name))

    plt.plot(preds_df["predictions"], preds_df["true values"],'bo')
    plt.plot([0,700000], [0,700000])
    # line1, = plt.plot(y_pre, Y_test, 'b', label='predictions')
    # plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    # plt.ylabel('Sales price')
    plt.xlabel('Predictions')
    plt.ylabel('true values')
    # plt.yscale("log")

    plt.show()
    
def displayGraphTrueVsPred2(name,y_pre,Y_test):
    plt.rcParams["figure.figsize"] = (17,4)

    print("Model: {}".format(name))

    preds_df=pd.DataFrame(data=y_pre,columns=["predictions"])
    preds_df["true values"]=Y_test
    preds_df.sort_values(by="true values",inplace=True)
    preds_df.reset_index(drop=True,inplace=True)


    line1, = plt.plot(preds_df["predictions"], 'b',linewidth=2)
    line2, = plt.plot(preds_df["true values"], 'c')
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=1)})
    plt.ylabel('Sales price')

    plt.show()


def displayErrGraphOnAllAlgorithms(containers):
    plt.rcParams["figure.figsize"] = (14,2)

    names=[(lambda row: row[0])(row) for row in containers]
    mean_squared_errs=[(lambda row: row[3])(row) for row in containers]

    errs_df=pd.DataFrame(data=mean_squared_errs,columns=["err"])
    errs_df["name"]=names
    errs_df.sort_values(by="err",inplace=True)

    line1, = plt.plot(errs_df["name"],errs_df["err"], 'b', label='Train f1 score on lying straight')
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=1)})
    plt.ylabel('Mean squared error')
    plt.xlabel('Model')
    plt.yscale("log")

    plt.show()

## Try on numerical features

In [ ]:
models = []
models.append(("SVC",SVC()))
models.append(("LinearSVC",LinearSVC()))
models.append(("KNeighbors",KNeighborsClassifier()))
models.append(("DecisionTree",DecisionTreeClassifier()))
models.append(("RandomForest",RandomForestClassifier()))

modelsWhichNeedNormalization = []
modelsWhichNeedNormalization.append(("LogisticRegression",LogisticRegression()))
modelsWhichNeedNormalization.append(("MLPClassifier",MLPClassifier(solver='lbfgs', random_state=0)))
modelsWhichNeedNormalization.append(("LinearRegression", LinearRegression()))


In [ ]:
if "Id" in X_numerical_not_missing_values_labels:
    X_numerical_not_missing_values_labels.remove("Id")
X_train,X_test,Y_train,Y_test = splitDataSet(dfTransformed,X_numerical_not_missing_values_labels)
containers_on_num_features = trainOnMultipleModels(X_train,X_test,Y_train,Y_test,models,modelsWhichNeedNormalization)
displayErrGraphOnAllAlgorithms(containers_on_num_features)

In [ ]:
print(Y_test[0:5])

print(containers_on_num_features[7][2][0:5].astype(int))
print(containers_on_num_features[2][2][0:5])
print(containers_on_num_features[4][2][0:5])

# displayGraphTrueVsPred(containers[7][0],containers[7][2],Y_test)
# displayGraphTrueVsPred(containers[2][0],containers[2][2],Y_test)
displayGraphTrueVsPred2(containers_on_num_features[7][0],containers_on_num_features[7][2],Y_test)
displayGraphTrueVsPred2(containers_on_num_features[2][0],containers_on_num_features[2][2],Y_test)
displayGraphTrueVsPred2(containers_on_num_features[4][0],containers_on_num_features[4][2],Y_test)

## Try on numerical and categorical features

In [ ]:
X_train,X_test,Y_train,Y_test = splitDataSet(dfTransformed,feature_columns)
containers_with_num_cat_features = trainOnMultipleModels(X_train,X_test,Y_train,Y_test,models,modelsWhichNeedNormalization)
displayErrGraphOnAllAlgorithms(containers_with_num_cat_features)

In [ ]:

# displayGraphTrueVsPred(containers[7][0],containers[7][2],Y_test)
# displayGraphTrueVsPred(containers[2][0],containers[2][2],Y_test)
displayGraphTrueVsPred2(containers_with_num_cat_features[7][0],containers_with_num_cat_features[7][2],Y_test)
displayGraphTrueVsPred2(containers_with_num_cat_features[2][0],containers_with_num_cat_features[2][2],Y_test)
displayGraphTrueVsPred2(containers_with_num_cat_features[4][0],containers_with_num_cat_features[4][2],Y_test)

## Conclusion

In [ ]:

displayErrGraphOnAllAlgorithms(containers_on_num_features)
displayErrGraphOnAllAlgorithms(containers_with_num_cat_features)
displayGraphTrueVsPred2(containers_on_num_features[7][0],containers_on_num_features[7][2],Y_test)
displayGraphTrueVsPred2(containers_on_num_features[2][0],containers_on_num_features[2][2],Y_test)